In [ ]:
from qdrant_client import QdrantClient

from src.collection.query_collection import (
    filter_search,
    get_semantically_similar_results,
)
from src.utils.utils import load_qdrant_client, load_config
from src.utils.utils import load_model

from dotenv import load_dotenv
import os
import pickle

load_dotenv()

QDRANT_HOST = os.getenv("QDRANT_HOST")
QDRANT_PORT = os.getenv("QDRANT_PORT")
COLLECTION_NAME = os.getenv("COLLECTION_NAME")
HF_MODEL_NAME = os.getenv("HF_MODEL_NAME")

In [ ]:
config = load_config("../.config/config.json")
similarity_threshold = float(config.get("similarity_threshold_1"))

with open("../data/regex_ids.pkl", "rb") as f:
    regex_ids = pickle.load(f)

In [ ]:
qdrant = load_qdrant_client(QDRANT_HOST, port=QDRANT_PORT)
model = load_model(HF_MODEL_NAME)

In [ ]:
query_embedding = model.encode("applications")

In [ ]:
results = get_semantically_similar_results(
    client=qdrant,
    collection_name=COLLECTION_NAME,
    query_embedding=query_embedding,
    score_threshold=0.1,
)

In [ ]:
results

In [ ]:
result_ids = [str(result.id) for result in results]
result_ids

In [ ]:
set(result_ids) & set(apps_ids)

In [ ]:
app_ids = regex_ids["application"]
apps_ids = regex_ids["applications"]

print(f"intersection: {sorted(list(set(app_ids) & set(apps_ids)))}")
print(f"apps_ids: {sorted(apps_ids)}")
print(f"count of app_ids: {len(app_ids)}")